# This notebook is inteed to generate the ground truth about the dataset

1. Read both databases in PySpark
2. Select 1000 random samples of each dataset. Those sample needs to be in english
3. Rewrite those 2k examples using Gemini-pro with a fixed prompt. Use langchang
4. Generate the vector embedding for those rewrited phrases.
5. Save both test datasets for each base

In [1]:
import os
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession

os.environ["SPARK_LOCAL_IP"] = "localhost"

conf = (
    SparkConf()
    .setMaster("local[*]")
    .setAppName("Delta Lake Spark Jupyter Lab")
    .set("spark.driver.memory", "30g")
    .set("spark.executor.memory", "30g")
    .set("spark.driver.cores", "16")
    .set("spark.worker.cores", "16")
    .set("spark.jars", "/home/baptvit/Documents/github/mineracao-dados-massivos/apps/jars/*")
    .set("spark.serializer", "org.apache.spark.serializer.KryoSerializer")
    .set(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
    .set(
        "spark.sql.delta.catalogImpl", "org.apache.spark.sql.delta.catalog.DeltaCatalog"
    )
)
sc = SparkContext(conf=conf).getOrCreate()
spark: SparkSession = SparkSession(sc)

24/06/19 16:59:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/19 16:59:00 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/19 16:59:00 ERROR SparkContext: Failed to add /home/baptvit/Documents/github/mineracao-dados-massivos/apps/jars/* to Spark environment
java.io.FileNotFoundException: Jar /home/baptvit/Documents/github/mineracao-dados-massivos/apps/jars/* not found
	at org.apache.spark.SparkContext.addLocalJarFile$1(SparkContext.scala:2100)
	at org.apache.spark.SparkContext.addJar(SparkContext.scala:2156)
	at org.apache.spark.SparkContext.$anonfun$new$15(SparkContext.scala:526)
	at org.apache.spark.SparkContext.$anonfun$new$15$adapted(SparkContext.scala:526)
	at scala.collection.mutable.ResizableArray.foreach(ResizableArray.scala:62

In [2]:
spark

## 1. Read both datasets

In [3]:
df_texbook = spark.read.parquet("/home/baptvit/Documents/github/mineracao-dados-massivos/data/med-qa-dataset/textbook_transfomed_parquet_parcioned_16")

In [5]:
df_texbook.show(5)

+--------------------------------+--------------------+--------------+--------------------+
|                        sentence|  embedding_sentence|token_sentence|            metadata|
+--------------------------------+--------------------+--------------+--------------------+
|(2）腺癌：分为：①原位腺癌（ad...|[-0.0382446125149...|           198|Raw file name is:...|
|            Major or mild neu...|[-0.3959405720233...|           108|Raw file name is:...|
|            The prevalence of...|[-0.1243198439478...|            23|Raw file name is:...|
|                     K. Jane Lee|[-0.0953831449151...|             6|Raw file name is:...|
|                  第六章心肌疾病|[-0.4195537567138...|             9|Raw file name is:...|
+--------------------------------+--------------------+--------------+--------------------+
only showing top 5 rows



In [9]:
df_texbook.count()

834392

In [7]:
df_questions = spark.read.parquet("/home/baptvit/Documents/github/mineracao-dados-massivos/data/med-qa-dataset/dataset/questions_transfomed_refat")

In [8]:
df_questions.show(5)

+--------------------+--------------------+--------------+--------------------+
|            sentence|  embedding_sentence|token_sentence|            metadata|
+--------------------+--------------------+--------------+--------------------+
|The questions is:...|[-0.4065425097942...|           325|Raw file name is:...|
|The questions is:...|[-0.4131570756435...|           308|Raw file name is:...|
|The questions is:...|[-0.3632318377494...|           297|Raw file name is:...|
|The questions is:...|[-0.2506562471389...|           201|Raw file name is:...|
|The questions is:...|[-0.3802637457847...|           327|Raw file name is:...|
+--------------------+--------------------+--------------+--------------------+
only showing top 5 rows



In [10]:
df_questions.count()

27040

## 2. Select 1000 random samples of each dataset. Those sample needs to be in english

### Textbook preprocess

In [18]:
seed=123456789

target_list_english = ['Gynecology_Novak.txt',
 'Pediatrics_Nelson.txt',
 'Pathology_Robbins.txt',
 'Pharmacology_Katzung.txt',
 'First_Aid_Step2.txt',
 'Biochemistry_Lippincott.txt',
 'Anatomy_Gray.txt',
 'Immunology_Janeway.txt',
 'Neurology_Adams.txt',
 'Cell_Biology_Alberts.txt',
 'Surgery_Schwartz.txt',
 'Pathoma_Husain.txt',
 'Obstentrics_Williams.txt',
 'First_Aid_Step1.txt',
 'Psichiatry_DSM-5.txt',
 'InternalMed_Harrison.txt',
 'Histology_Ross.txt',
 'Physiology_Levy.txt']

In [22]:
for textbook in target_list_english:
    print(f"(df_texbook.metadata.contains('{textbook}'))")
    print("|")

(df_texbook.metadata.contains('Gynecology_Novak.txt'))
|
(df_texbook.metadata.contains('Pediatrics_Nelson.txt'))
|
(df_texbook.metadata.contains('Pathology_Robbins.txt'))
|
(df_texbook.metadata.contains('Pharmacology_Katzung.txt'))
|
(df_texbook.metadata.contains('First_Aid_Step2.txt'))
|
(df_texbook.metadata.contains('Biochemistry_Lippincott.txt'))
|
(df_texbook.metadata.contains('Anatomy_Gray.txt'))
|
(df_texbook.metadata.contains('Immunology_Janeway.txt'))
|
(df_texbook.metadata.contains('Neurology_Adams.txt'))
|
(df_texbook.metadata.contains('Cell_Biology_Alberts.txt'))
|
(df_texbook.metadata.contains('Surgery_Schwartz.txt'))
|
(df_texbook.metadata.contains('Pathoma_Husain.txt'))
|
(df_texbook.metadata.contains('Obstentrics_Williams.txt'))
|
(df_texbook.metadata.contains('First_Aid_Step1.txt'))
|
(df_texbook.metadata.contains('Psichiatry_DSM-5.txt'))
|
(df_texbook.metadata.contains('InternalMed_Harrison.txt'))
|
(df_texbook.metadata.contains('Histology_Ross.txt'))
|
(df_texbook.met

In [11]:
seed=123456789

In [19]:
from pyspark.sql.functions import col

In [26]:
df_textbook_english = df_texbook.filter((df_texbook.metadata.contains('Gynecology_Novak.txt'))
|
(df_texbook.metadata.contains('Pediatrics_Nelson.txt'))
|
(df_texbook.metadata.contains('Pathology_Robbins.txt'))
|
(df_texbook.metadata.contains('Pharmacology_Katzung.txt'))
|
(df_texbook.metadata.contains('First_Aid_Step2.txt'))
|
(df_texbook.metadata.contains('Biochemistry_Lippincott.txt'))
|
(df_texbook.metadata.contains('Anatomy_Gray.txt'))
|
(df_texbook.metadata.contains('Immunology_Janeway.txt'))
|
(df_texbook.metadata.contains('Neurology_Adams.txt'))
|
(df_texbook.metadata.contains('Cell_Biology_Alberts.txt'))
|
(df_texbook.metadata.contains('Surgery_Schwartz.txt'))
|
(df_texbook.metadata.contains('Pathoma_Husain.txt'))
|
(df_texbook.metadata.contains('Obstentrics_Williams.txt'))
|
(df_texbook.metadata.contains('First_Aid_Step1.txt'))
|
(df_texbook.metadata.contains('Psichiatry_DSM-5.txt'))
|
(df_texbook.metadata.contains('InternalMed_Harrison.txt'))
|
(df_texbook.metadata.contains('Histology_Ross.txt'))
|
(df_texbook.metadata.contains('Physiology_Levy.txt'))).filter((df_texbook.token_sentence > 200) & (df_texbook.token_sentence < 512))

In [30]:
## https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.sample.html
df_textbook_english_test = df_textbook_english.sample(False, 0.1, seed=seed).limit(1000)

In [31]:
df_textbook_english_test.show()

+--------------------+--------------------+--------------+--------------------+
|            sentence|  embedding_sentence|token_sentence|            metadata|
+--------------------+--------------------+--------------+--------------------+
|The essential fea...|[-0.2756949663162...|           210|Raw file name is:...|
|A variety of toxi...|[-0.3810059428215...|           245|Raw file name is:...|
|Mitral valve prol...|[-0.3140565752983...|           279|Raw file name is:...|
|1378 parasites (i...|[-0.2810960412025...|           399|Raw file name is:...|
|Physicians should...|[-0.2339465767145...|           246|Raw file name is:...|
|For the most part...|[-0.2888492643833...|           236|Raw file name is:...|
|FIguRE 69-1 Role ...|[-0.1749157905578...|           264|Raw file name is:...|
|96e-8 The U.S. Na...|[-0.2746782898902...|           209|Raw file name is:...|
|Cerebral Perfusio...|[-0.3238443136215...|           404|Raw file name is:...|
|(e.g., pregnancy ...|[-0.2209758758544.

In [32]:
df_textbook_english_test.count()

1000

### Question answering preprocess

In [33]:
df_questions_filtered = df_questions.sample(False, 0.1, seed=seed).limit(1000)

In [34]:
df_questions_filtered.show()

+--------------------+--------------------+--------------+--------------------+
|            sentence|  embedding_sentence|token_sentence|            metadata|
+--------------------+--------------------+--------------+--------------------+
|The questions is:...|[-0.4065425097942...|           325|Raw file name is:...|
|The questions is:...|[-0.3099440634250...|           150|Raw file name is:...|
|The questions is:...|[-0.3393301069736...|           142|Raw file name is:...|
|The questions is:...|[-0.3057477474212...|           238|Raw file name is:...|
|The questions is:...|[-0.2709197700023...|           322|Raw file name is:...|
|The questions is:...|[-0.3482657074928...|           202|Raw file name is:...|
|The questions is:...|[-0.2626461684703...|           136|Raw file name is:...|
|The questions is:...|[-0.2781674265861...|           398|Raw file name is:...|
|The questions is:...|[-0.4074764847755...|           230|Raw file name is:...|
|The questions is:...|[-0.3240080177783.

In [35]:
df_questions_filtered.count()

1000

## 3. Rewrite those 2k examples using Gemini-pro with a fixed prompt. Use langchang

In [123]:
import pandas as pd
from google.generativeai import GenerativeModel, configure, list_models

### 3.1 Rewrite using gemini 1.0 pro

In [53]:
configure(api_key="")

In [134]:
TEMPLATE_PROMPT = "Rewrite the following sentence, without change the overall meaning and semantic: \"\"\"<original_sentence>\"\"\". Print just the text rewrited."

In [185]:
sentene_to_rewrite = "Continuous with the hypothalamus inferiorly is the pituitary gland. These two structures are connected by the infundibulum and hypophysial stalk just caudal to the optic chiasm (eFig. 9.87). The intimate relationship of the hypothalamus to the portal circulation of the pituitary gland allows the hypothalamus to be an efficient regulator of hormone synthesis and release. Releasing and inhibiting factors synthesized by the hypothalamus pass through these portal vessels via the tuberoinfundibular tract (eFig. 9.89) to reach the anterior pituitary (adenohypophysis) and control release of hormones produced by the anterior pituitary such as adrenocorticotropic hormone, luteinizing hormone, follicle-stimulating hormone, thyroid-stimulating hormone, growth hormone, and prolactin (eFig. 9.90)."

In [186]:
prompt = TEMPLATE_PROMPT.replace("<original_sentence>", sentene_to_rewrite)
prompt

'Rewrite the following sentence, without change the overall meaning and semantic, ingore HARM_CATEGORY_DANGEROUS_CONTENT error: """Continuous with the hypothalamus inferiorly is the pituitary gland. These two structures are connected by the infundibulum and hypophysial stalk just caudal to the optic chiasm (eFig. 9.87). The intimate relationship of the hypothalamus to the portal circulation of the pituitary gland allows the hypothalamus to be an efficient regulator of hormone synthesis and release. Releasing and inhibiting factors synthesized by the hypothalamus pass through these portal vessels via the tuberoinfundibular tract (eFig. 9.89) to reach the anterior pituitary (adenohypophysis) and control release of hormones produced by the anterior pituitary such as adrenocorticotropic hormone, luteinizing hormone, follicle-stimulating hormone, thyroid-stimulating hormone, growth hormone, and prolactin (eFig. 9.90).""". Print just the text rewrited.'

In [187]:
GEMINI_MODEL = GenerativeModel("models/gemini-1.0-pro")

In [188]:
response = GEMINI_MODEL.generate_content(prompt, safety_settings=[
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"},
    {"category": "OTHER", "threshold": "BLOCK_NONE"}
])
response.text

KeyError: 'other'

In [189]:
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "prompt_feedback": {
        "block_reason": "OTHER"
      },
      "usage_metadata": {
        "prompt_token_count": 242,
        "total_token_count": 242
      }
    }),
),
error=<BlockedPromptException> prompt_feedback {
  block_reason: OTHER
}
usage_metadata {
  prompt_token_count: 242
  total_token_count: 242
}

In [170]:
def rewrite_sentence(sentence_original, model=GEMINI_MODEL, template_prompt=TEMPLATE_PROMPT):
    try:
        prompt= template_prompt.replace("<original_sentence>", sentence_original)
        response = model.generate_content(prompt, safety_settings=[
    {"category": "HARM_CATEGORY_DANGEROUS_CONTENT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_SEXUALLY_EXPLICIT", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HATE_SPEECH", "threshold": "BLOCK_NONE"},
    {"category": "HARM_CATEGORY_HARASSMENT", "threshold": "BLOCK_NONE"}
])
        return response.text
    except Exception as e:
        return "Error text"

### 3.2 Generate vector_embedding

In [200]:
from embedding_model.sentence_embedding_bert_model import BertSentenceEmbedding

BERT_MODEL = BertSentenceEmbedding()

In [73]:
embedding_dict = model.get_sentence_embedding(response.text)

Insufficient reputation prevents me from commenting, so I provide my response here. My environment includes Python 3.11.5 and Notebook 7.0.4, where I encountered the described error.
Shape of Sentence Embeddings: torch.Size([1, 768])
40


In [78]:
#embedding_dict["embedding_sentence"]
embedding_dict["token_sentence"]

40

In [201]:
def vector_embedding_token(sentence, model=BERT_MODEL):
    embedding_dict = model.get_sentence_embedding(sentence)
    return embedding_dict["embedding_sentence"], embedding_dict["token_sentence"]

### 3.3 Aplly on the textbook test dataset

In [84]:
pd_textbook_test = df_textbook_english_test.toPandas()

In [87]:
pd_textbook_test

,sentence,embedding_sentence,token_sentence,metadata
0,The essential feature of enuresis is repeated ...,"[-0.27569496631622314, 0.13381759822368622, 0....",210,"Raw file name is: Psichiatry_DSM-5.txt, Were r..."
1,A variety of toxic ingestions cause a metaboli...,"[-0.3810059428215027, 0.3697296977043152, 0.16...",245,"Raw file name is: Pediatrics_Nelson.txt, Were ..."
2,"Mitral valve prolapse, in which one leaflet mo...","[-0.3140565752983093, -0.22627578675746918, 0....",279,"Raw file name is: InternalMed_Harrison.txt, We..."
3,"1378 parasites (i.e., >20% of parasites with v...","[-0.28109604120254517, 0.16703666746616364, -0...",399,"Raw file name is: InternalMed_Harrison.txt, We..."
4,Physicians should suspect C. sordellii infecti...,"[-0.23394657671451569, 0.11328402906656265, 0....",246,"Raw file name is: InternalMed_Harrison.txt, We..."
...,...,...,...,...
995,The relation of estrogen therapy to cancer con...,"[-0.448002427816391, 0.0888386145234108, 0.066...",334,"Raw file name is: Pharmacology_Katzung.txt, We..."
996,Operating Room Safety The operating room is by...,"[-0.3043455481529236, 0.15406396985054016, 0.1...",239,"Raw file name is: Gynecology_Novak.txt, Were r..."
997,FIGURE 31-2 Percentage of births with trial of...,"[-0.36733120679855347, -0.1072595864534378, 0....",232,"Raw file name is: Obstentrics_Williams.txt, We..."
998,As a result of electronic access to medical in...,"[-0.42454156279563904, 0.40213125944137573, 0....",233,"Raw file name is: Gynecology_Novak.txt, Were r..."


In [101]:
pd_textbook_test["rewrited_sentence"] = pd_textbook_test.apply(lambda row: rewrite_sentence(row['sentence']), axis=1)

In [143]:
pd_textbook_test_error = pd_textbook_test[pd_textbook_test.rewrited_sentence == "Error text"]

In [144]:
pd_textbook_test_cool = pd_textbook_test[pd_textbook_test.rewrited_sentence != "Error text"]

In [154]:
pd_textbook_test_error["rewrited_sentence"] = pd_textbook_test_error.apply(lambda row: rewrite_sentence(row['sentence']), axis=1)

/tmp/ipykernel_155733/3263910005.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pd_textbook_test_error["rewrited_sentence"] = pd_textbook_test_error.apply(lambda row: rewrite_sentence(row['sentence']), axis=1)


In [158]:
df_correct1 = pd_textbook_test_error[pd_textbook_test_error.rewrited_sentence != "Error text"]

In [159]:
df_erro1 = pd_textbook_test_error[pd_textbook_test_error.rewrited_sentence == "Error text"]

In [171]:
df_erro1["rewrited_sentence"] = df_erro1.apply(lambda row: rewrite_sentence(row['sentence']), axis=1)

/tmp/ipykernel_155733/1935862757.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_erro1["rewrited_sentence"] = df_erro1.apply(lambda row: rewrite_sentence(row['sentence']), axis=1)


In [173]:
df_correct2 = df_erro1[df_erro1.rewrited_sentence != "Error text"]

In [174]:
df_erro2 = df_erro1[df_erro1.rewrited_sentence == "Error text"]

In [184]:
df_erro2

,sentence,embedding_sentence,token_sentence,metadata,rewrited_sentence
282,The diagnostic criteria for pedophilic disorde...,"[-0.1525697410106659, 0.28781938552856445, 0.1...",222,"Raw file name is: Psichiatry_DSM-5.txt, Were r...",Error text
576,Continuous with the hypothalamus inferiorly is...,"[0.08356308937072754, 0.17696286737918854, 0.2...",206,"Raw file name is: Anatomy_Gray.txt, Were read ...",Error text
711,"Haragan AF, Hulsey TC, Hawk AF, et al: Diagnos...","[-0.40991631150245667, -0.058627255260944366, ...",295,"Raw file name is: Obstentrics_Williams.txt, We...",Error text
876,Panic or anxiety can occur in association with...,"[-0.2736693322658539, 0.4352610409259796, -0.0...",270,"Raw file name is: Psichiatry_DSM-5.txt, Were r...",Error text


In [175]:
df_erro2.to_csv("teste.csv")

In [194]:
## https://spark.apache.org/docs/3.1.2/api/python/reference/api/pyspark.sql.DataFrame.sample.html
df_textbook_4 = df_textbook_english_test = df_textbook_english.sample(False, 0.1, seed=11).limit(4).toPandas()

In [195]:
df_textbook_4["rewrited_sentence"] = df_textbook_4.apply(lambda row: rewrite_sentence(row['sentence']), axis=1)

In [196]:
df_textbook_4

,sentence,embedding_sentence,token_sentence,metadata,rewrited_sentence
0,The essential feature of enuresis is repeated ...,"[-0.27569496631622314, 0.13381759822368622, 0....",210,"Raw file name is: Psichiatry_DSM-5.txt, Were r...","Enuresis involves recurrent, unintentional (or..."
1,A severe congenital form of DM can appear in i...,"[-0.3360024094581604, 0.1881510466337204, 0.10...",215,"Raw file name is: Pediatrics_Nelson.txt, Were ...",Severe congenital DM can arise in infants of m...
2,"Thus, individuals may search for a particular ...","[-0.21101263165473938, -0.09675713628530502, 0...",251,"Raw file name is: Psichiatry_DSM-5.txt, Were r...",Individuals may have a specific scab they sear...
3,"Before speaking with a child about death, the ...","[-0.27277547121047974, 0.33284392952919006, 0....",416,"Raw file name is: Pediatrics_Nelson.txt, Were ...",**Rewritten Text:**\n\nBefore discussing death...


In [163]:
df_erro1.to_csv("teste.csv")

In [197]:
# Union two dataframes 
frames = [pd_textbook_test_cool, df_correct1, df_correct2, df_textbook_4]
result = pd.concat(frames)

In [198]:
result

,sentence,embedding_sentence,token_sentence,metadata,rewrited_sentence
0,The essential feature of enuresis is repeated ...,"[-0.27569496631622314, 0.13381759822368622, 0....",210,"Raw file name is: Psichiatry_DSM-5.txt, Were r...",Enuresis is characterized by repeated involunt...
2,"Mitral valve prolapse, in which one leaflet mo...","[-0.3140565752983093, -0.22627578675746918, 0....",279,"Raw file name is: InternalMed_Harrison.txt, We...","Mitral valve prolapse, where one leaflet shift..."
3,"1378 parasites (i.e., >20% of parasites with v...","[-0.28109604120254517, 0.16703666746616364, -0...",399,"Raw file name is: InternalMed_Harrison.txt, We...",1378 parasites in peripheral blood or phagocyt...
4,Physicians should suspect C. sordellii infecti...,"[-0.23394657671451569, 0.11328402906656265, 0....",246,"Raw file name is: InternalMed_Harrison.txt, We...",Suspect C. sordellii infection in afebrile pat...
5,"For the most part, the treatment of hemoptysis...","[-0.28884926438331604, 0.24810218811035156, 0....",236,"Raw file name is: InternalMed_Harrison.txt, We...",Hemoptysis treatment typically depends on the ...
...,...,...,...,...,...
974,The most widespread use of acyclovir is in the...,"[-0.48253560066223145, 0.017199335619807243, -...",368,"Raw file name is: InternalMed_Harrison.txt, We...",Acyclovir is widely used to treat genital herp...
0,The essential feature of enuresis is repeated ...,"[-0.27569496631622314, 0.13381759822368622, 0....",210,"Raw file name is: Psichiatry_DSM-5.txt, Were r...","Enuresis involves recurrent, unintentional (or..."
1,A severe congenital form of DM can appear in i...,"[-0.3360024094581604, 0.1881510466337204, 0.10...",215,"Raw file name is: Pediatrics_Nelson.txt, Were ...",Severe congenital DM can arise in infants of m...
2,"Thus, individuals may search for a particular ...","[-0.21101263165473938, -0.09675713628530502, 0...",251,"Raw file name is: Psichiatry_DSM-5.txt, Were r...",Individuals may have a specific scab they sear...


In [202]:
result["rewrited_sentence_embedding"] = result.apply(lambda row: vector_embedding_token(row['rewrited_sentence']), axis=1)

Enuresis is characterized by repeated involuntary or intentional urination into bed or clothing (Criterion A). For a diagnosis, this urination must cause significant distress or impairment in daily life (Criterion B). Individuals must be of an age (chronological or mental age of 5 years) when continence is expected (Criterion C). Enuresis is not caused by medical conditions or substance use (Criterion D).
Shape of Sentence Embeddings: torch.Size([1, 768])
87
Mitral valve prolapse, where one leaflet shifts behind the plane of the other, can arise from myxomatous valve degeneration, excessive leaflet tissue, impaired chordal structures due to degeneration, or papillary muscle rupture or dysfunction post-myocardial infarction. Color flow Doppler reveals regurgitant jets, with velocity dictated by the pressure gradient between the chambers. This velocity is typically high in left-sided regurgitant lesions, such as mitral or aortic regurgitation, resulting in turbulent jets on color flow Do

In [203]:
result.to_csv("pd_textbook_test_embedding_1000.csv")

In [ ]:
teste = result.rewrited_sentence_embedding: pd.Series

0      ([[-0.08195801079273224, 0.1827828586101532, 0...
2      ([[-0.24928240478038788, -0.21157097816467285,...
3      ([[-0.2917247712612152, 0.11056926101446152, -...
4      ([[-0.2776722311973572, 0.10505430400371552, 0...
5      ([[-0.2406291663646698, 0.29498544335365295, 0...
                             ...                        
974    ([[-0.40039151906967163, -0.003660049987956881...
0      ([[-0.07649222761392593, 0.15640617907047272, ...
1      ([[-0.36263635754585266, 0.17458859086036682, ...
2      ([[-0.21761690080165863, 0.07891286909580231, ...
3      ([[-0.26748591661453247, 0.45752763748168945, ...
Name: rewrited_sentence_embedding, Length: 1000, dtype: object

In [ ]:
teste|